In [1]:
!pip install requests

In [6]:
import requests
from urllib import parse
import pandas as pd

In [7]:
### API 요청 (본인의 키 사용)

In [8]:
# API 요청.(본인의 API 키 사용)
def geocode(address, type='ROAD'):
    result = requests.get("http://api.vworld.kr/req/address?service=address"
        + "&request=getcoord" +"&version2.0" + "&crs=epsg:4326"
        + "&address=" + parse.quote(address) + "&type=" + type
        + "&key=D84AD2C2-7DA5-3FE3-BB6C-64F509314F5D")
    return result




In [9]:
#https://api.vworld.kr/req/address?service=address
#    + &request=getcoord&version=2.0&crs=epsg:4326
#        &address=%ED%9A%A8%EB%A0%B9%EB%A1%9C72%EA%B8%B8%2060
#        &refine=true&simple=false&format=xml&type=road&key=[KEY]

In [10]:
CB_file = pd.read_csv('CoffeeBean_2024_2.csv',encoding='cp949',  engine='python')

CB_file.head() #작업 내용 확인용 출력

,store,address,phone,address2
0,차병원점,서울시 강남구 논현로 566 강남차병원1층,02-538-7615,서울특별시 강남구 논현로 566 강남차병원1층
1,강남대로점,서울시 서초구 강남대로 369 1층,02-588-5778,서울특별시 서초구 강남대로 369 1층
2,청담에스점,"서울시 강남구 압구정로 461 네이처포엠빌딩B108,109호",02-548-6052,"서울특별시 강남구 압구정로 461 네이처포엠빌딩B108,109호"
3,신사점,서울시 강남구 도산대로 126,02-548-2741,서울특별시 강남구 도산대로 126
4,역삼점,"서울시 강남구 논현로 512 지상1,2층",02-569-8051,"서울특별시 강남구 논현로 512 지상1,2층"


In [11]:
# 경도, 위도 필드 만들기
CB_file['경도'] = 0.0
CB_file['위도'] = 0.0
CB_file

,store,address,phone,address2,경도,위도
0,차병원점,서울시 강남구 논현로 566 강남차병원1층,02-538-7615,서울특별시 강남구 논현로 566 강남차병원1층,0.0,0.0
1,강남대로점,서울시 서초구 강남대로 369 1층,02-588-5778,서울특별시 서초구 강남대로 369 1층,0.0,0.0
2,청담에스점,"서울시 강남구 압구정로 461 네이처포엠빌딩B108,109호",02-548-6052,"서울특별시 강남구 압구정로 461 네이처포엠빌딩B108,109호",0.0,0.0
3,신사점,서울시 강남구 도산대로 126,02-548-2741,서울특별시 강남구 도산대로 126,0.0,0.0
4,역삼점,"서울시 강남구 논현로 512 지상1,2층",02-569-8051,"서울특별시 강남구 논현로 512 지상1,2층",0.0,0.0
...,...,...,...,...,...,...
116,상암한국지역정보개발원점,서울시 마포구 성암로301 KLID한국지역정보개발원 1층,02-376-9162,서울특별시 마포구 성암로301 KLID한국지역정보개발원 1층,0.0,0.0
117,분당정자점,경기도 성남시 분당구 정자일로 227(정자동) 백궁지엔느주차전용빌딩 1층,031-786-1906,경기도 성남시 분당구 정자일로 227(정자동) 백궁지엔느주차전용빌딩 1층,0.0,0.0
118,신논현역점,서울특별시 강남구 봉은사로108 글라스톤빌딩1층,02-569-9061,서울특별시 강남구 봉은사로108 글라스톤빌딩1층,0.0,0.0
119,강서그랜드아이파크점,서울시 강서구 화곡6동 1095번지 1층,02-2699-7462,서울특별시 강서구 화곡6동 1095번지 1층,0.0,0.0


In [12]:
# 각 주소마다 API 요청
for i in range(len(CB_file)):
    result = None
    coord_x = coord_y = 0.0
    result = geocode(CB_file.at[i, 'address2']).json()
    if result['response']['status'] != 'NOT_FOUND':
        coord_x = result['response']['result']['point']['x']
        coord_y = result['response']['result']['point']['y']
    else:    # 도로명 주소로 못 찾을 시 지번 주소로 찾기
        result = geocode(CB_file.at[i, 'address2'], 'PARCEL').json()
        if result['response']['status'] != 'NOT_FOUND':
            coord_x = result['response']['result']['point']['x']
            coord_y = result['response']['result']['point']['y']
        else: 
            continue
    
    CB_file.at[i, '경도'] = round(float(coord_x), 4)
    CB_file.at[i, '위도'] = round(float(coord_y), 4)



In [13]:
CB_file

,store,address,phone,address2,경도,위도
0,차병원점,서울시 강남구 논현로 566 강남차병원1층,02-538-7615,서울특별시 강남구 논현로 566 강남차병원1층,127.0345,37.5068
1,강남대로점,서울시 서초구 강남대로 369 1층,02-588-5778,서울특별시 서초구 강남대로 369 1층,127.0282,37.4957
2,청담에스점,"서울시 강남구 압구정로 461 네이처포엠빌딩B108,109호",02-548-6052,"서울특별시 강남구 압구정로 461 네이처포엠빌딩B108,109호",127.0472,37.5252
3,신사점,서울시 강남구 도산대로 126,02-548-2741,서울특별시 강남구 도산대로 126,127.0224,37.5170
4,역삼점,"서울시 강남구 논현로 512 지상1,2층",02-569-8051,"서울특별시 강남구 논현로 512 지상1,2층",127.0365,37.5023
...,...,...,...,...,...,...
116,상암한국지역정보개발원점,서울시 마포구 성암로301 KLID한국지역정보개발원 1층,02-376-9162,서울특별시 마포구 성암로301 KLID한국지역정보개발원 1층,0.0000,0.0000
117,분당정자점,경기도 성남시 분당구 정자일로 227(정자동) 백궁지엔느주차전용빌딩 1층,031-786-1906,경기도 성남시 분당구 정자일로 227(정자동) 백궁지엔느주차전용빌딩 1층,127.1056,37.3707
118,신논현역점,서울특별시 강남구 봉은사로108 글라스톤빌딩1층,02-569-9061,서울특별시 강남구 봉은사로108 글라스톤빌딩1층,0.0000,0.0000
119,강서그랜드아이파크점,서울시 강서구 화곡6동 1095번지 1층,02-2699-7462,서울특별시 강서구 화곡6동 1095번지 1층,0.0000,0.0000


In [14]:
CB_file.to_csv('CB_geoAPI.shp_2024_2.csv',encoding='CP949', index = False)